In [1]:
from rank_bm25 import BM25Okapi
from tqdm import tqdm
import utils
from transformers import AutoTokenizer
model_name = "/home/zhoushiqi/workplace/model/deepseek-coder-6.7b-instruct"  # 你可以换成任何Hugging Face模型名称

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

datasets = utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/megadiff-single-function/process.jsonl")
corpus = [data['diff_context'] for data in datasets]
tokenize_corpus = [data['tokenize_diff_context'] for data in datasets]


bm25 = BM25Okapi(tokenize_corpus)

buggy = utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/df4_process_data/one_function/1.2.jsonl")
answer = utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/evaluate_results/deepseek/baseline_1.2_N100_T1_process.jsonl")[0]


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
answer[b1['project'] + '_' + str(b1['bug_id'])][50]

'\n    public void atan2(final double[] y, final int yOffset,\n                      final double[] x, final int xOffset,\n                      final double[] result, final int resultOffset) {\n\n        // compute r = sqrt(x^2+y^2)\n        double[] tmp1 = new double[getSize()];\n        multiply(x, xOffset, x, xOffset, tmp1, 0);      // x^2\n        double[] tmp2 = new double[getSize()];\n        multiply(y, yOffset, y, yOffset, tmp2, 0);      // y^2\n        add(tmp1, 0, tmp2, 0, tmp2, 0);                 // x^2 + y^2\n        rootN(tmp2, 0, 2, tmp1, 0);                     // r = sqrt(x^2 + y^2)\n\n        if (x[xOffset] >= 0) {\n\n            // compute atan2(y, x) = 2 atan(y / (r + x))\n            add(tmp1, 0, x, xOffset, tmp2, 0);          // r + x\n            divide(y, yOffset, tmp2, 0, tmp1, 0);       // y /(r + x)\n            atan(tmp1, 0, tmp2, 0);                     // atan(y / (r + x))\n            for (int i = 0; i < tmp2.length; ++i) {\n                result[result

In [20]:
from collections import defaultdict
import tqdm
bug_id = 0
all_retrieval = defaultdict(int)
for i in tqdm.tqdm(range(100)):
    fix_id = i
    b1 = buggy[bug_id]
    a1 = answer[b1['project'] + '_' + str(b1['bug_id'])][fix_id]
    data = {'buggy_function':b1['erro_repairs'][0]['src_code'][0], 'fixed_function':a1.strip('\n')}
    query = utils._process(data, 3)
    tokenized_query = tokenizer.tokenize(query)
    retrive = bm25.get_top_n(tokenized_query, corpus, n=10)
    for r in retrive:
        all_retrieval[r] += 1

100%|██████████| 100/100 [28:53<00:00, 17.34s/it]


In [22]:
order = sorted(all_retrieval, key=lambda x:all_retrieval[x], reverse=True)

In [3]:

from tqdm import tqdm
import utils
from transformers import AutoTokenizer
model_name = "/home/zhoushiqi/workplace/model/deepseek-coder-6.7b-instruct"  # 你可以换成任何Hugging Face模型名称

# 加载tokenizerfrom gensim_bm25 import bm25
tokenizer = AutoTokenizer.from_pretrained(model_name)

datasets = utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/megadiff-single-function/process.jsonl")
documents = [data['buggy_function'] for data in datasets]


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import tqdm
new_datasets = []
index = 0
for data in tqdm(datasets):
    new_data = {}
    b = data["buggy_function"]
    b = b.strip()
    b = b.replace('\t', '    ')
    tokenized_buggy_code = tokenizer.tokenize(b)
    if len(tokenized_buggy_code) > 10000:
        continue
    f = data["fixed_function"]
    f = f.strip()
    f = f.replace('\t', '    ')
    tokenized_fixed_code = tokenizer.tokenize(f)
    new_data['buggy_function'] = b
    new_data['fixed_function'] = f
    new_data['diff_context'] = data["diff_context"]
    new_data['tokenized_diff_context'] = data["tokenize_diff_context"]
    new_data['tokenized_buggy_function'] = tokenized_buggy_code
    new_data['tokenized_fixed_function'] = tokenized_fixed_code
    new_datasets.append(new_data)

NameError: name 'datasets' is not defined

In [ ]:
utils.write_jsonl("/home/zhoushiqi/workplace/apr/data/megadiff-single-function/process.jsonl", new_datasets)

In [16]:
buggy = utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/df4_process_data/one_function/1.2.jsonl")
answer = utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/evaluate_results/deepseek/retrieval/baseline_1.2_OnlyBuggycode_N100_T1_process.jsonl")[0]

In [17]:
diff_datas = []
for bug in tqdm.tqdm(buggy):
    new_data = {}
    bug_id = f"{bug['project']}_{bug['bug_id']}"
    if bug_id in plausible_ids or bug_id not in answer:
        continue
    buggy_code = bug['erro_repairs'][0]['src_code'][0].strip()
    new_data['project'] = bug['project']
    new_data['bug_id'] = bug['bug_id']
    new_data['buggy_code'] = buggy_code
    new_data['fixed'] = [a.strip() for a in answer[bug_id]]
    new_data['diffs'] = [utils._process({'buggy_function':buggy_code, 'fixed_function':a}, 3) for a in new_data['fixed']]
    diff_datas.append(new_data)

100%|██████████| 255/255 [00:13<00:00, 18.41it/s]


In [1]:
import json
x = json.load(open("/home/zhoushiqi/workplace/minicpm/MiniCPM/finetune/data/AdvertiseGen/all.json"))

In [6]:
import utils
# y1 = utils.read_jsonl("/home/guochuanzhe/share/math_sft/MathInstruct/MathInstruct.jsonl")
# y2 = utils.read_jsonl("/home/guochuanzhe/share/math_sft/MetaMathQA/MetaMathQA.jsonl")
y3 = utils.read_jsonl("/home/guochuanzhe/share/math_sft/orca-math-word-problems-200k/orca-math-word-problems-200k.jsonl")
# y4 = utils.read_jsonl("/home/guochuanzhe/share/math_sft/reasoningdata_200k-sharegpt/reasoningdata_200k-sharegpt.jsonl")

In [10]:
import random
random.seed(42)
add = random.sample(y3, k=len(x)*2//8)

In [12]:
x += add

In [15]:
json.dump(x, open("/home/zhoushiqi/workplace/minicpm/MiniCPM/finetune/data/AdvertiseGen/all_add_math_8&2.json", 'w'))

In [19]:
json.dump(x, open("/home/zhoushiqi/workplace/minicpm/MiniCPM/finetune/data/AdvertiseGen/all_add_math.json", 'w'))

In [2]:
import utils
len(utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/evaluate_results/deepseek/retrieval/baseline_1.2_random_N200_T1_process.jsonl")[0])

223

In [34]:
import numpy as np
from gensim.similarities.docsim import MatrixSimilarity
# 创建一些简单的文档向量
documents_vectors = [[(0,0.121),(1,0.2)],[(0,0.2),(1,0.2)]]

# 创建相似度索引
index = MatrixSimilarity(documents_vectors, num_features=2)

# 创建一个简单的查询向量
query_vector = [(0,0.1),(1,0.1)]

# 执行查询
sims = index[query_vector]

# 输出查询结果
print(sims)
index.save("temp")


[0.9710257  0.99999994]


In [39]:
import utils
for i,x in enumerate(utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/megadiff-single-function/process_filtered2048.jsonl")):
    assert i == x['index']

In [36]:
import utils
x = utils.read_jsonl("/home/zhoushiqi/workplace/apr/data/evaluate_results/deepseek/retrieval/baseline_1.2_diff_vec_k2_N100_T1_result.jsonl")
t = 0
for xx in x:
    if "Failing tests: 0\n" in xx['result'][:100]:
        t += 1
t

17

In [374]:
len(x)

187

In [11]:
import json
y = json.load(open("/home/zhoushiqi/workplace/apr/data/evaluate_results/deepseek/retrieval/baseline_1.2_diff_vec_k1_N100_T1_process.jsonl"))

In [340]:
for yy in y.values():
    if '' in yy:
        print(1)

In [12]:
set(y.keys())-(set(all) - set(plausible))

{'Closure_128', 'Closure_2'}

In [25]:
other = set(plausible).intersection(set(x_k))

In [30]:
set(all) - (set(x_k)-other).union(set(plausible))

{'Lang_43'}

In [34]:
m = {"project": "Lang", "bug_id": 43, "result": ["Failing tests: 1\n  - org.apache.commons.lang.text.ExtendedMessageFormatTest::testEscapedQuote_LANG_477\n", "Failing tests: 1\n  - org.apache.commons.lang.text.ExtendedMessageFormatTest::testEscapedQuote_LANG_477\n", "Failing tests: 1\n  - org.apache.commons.lang.text.ExtendedMessageFormatTest::testEscapedQuote_LANG_477\n", "Failing tests: 1\n  - org.apache.commons.lang.text.ExtendedMessageFormatTest::testEscapedQuote_LANG_477\n", "Compile Failing", "Compile Failing", "Failing tests: 1\n  - org.apache.commons.lang.text.ExtendedMessageFormatTest::testEscapedQuote_LANG_477\n", "Failing tests: 1\n  - org.apache.commons.lang.text.ExtendedMessageFormatTest::testEscapedQuote_LANG_477\n", "Failing tests: 1\n  - org.apache.commons.lang.text.ExtendedMessageFormatTest::testEscapedQuote_LANG_477\n", "Failing tests: 1\n  - org.apache.commons.lang.text.ExtendedMessageFormatTest::testEscapedQuote_LANG_477\n"]}
len(m['result'])

10